<a href="https://colab.research.google.com/github/mackenziecoushayrichter/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col

# Read in the Review dataset as a DataFrame
df=df.dropna()
df=df.dropDuplicates()

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes','vine', 'verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100EO2AQP6B02|          3|            2|          4|   N|                Y|
|R100IITU077KKZ|          5|            0|          0|   N|                Y|
|R100WIAVSCW4D1|          5|            3|          5|   N|                N|
|R1023NXBAJ0BYT|          5|            0|          0|   N|                Y|
|R102CEVVIBXNFB|          1|            0|          2|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
#filter the data for just rows that have 20 or more votes
filter_df= vine_df.filter(vine_df.total_votes>=20)
filter_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20A1J42RKSO1I|          1|           97|        105|   N|                N|
| R6HXKDRL0KIE4|          1|           45|         52|   N|                Y|
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
| R4UPLCUG1KGSA|          5|           25|         27|   N|                Y|
|R107YKUN3UFA9D|          5|           34|         36|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
#filter the data for the rows where helpful votes/total votes is greater or equal to 50%
filter_df_2=filter_df.filter(filter_df.helpful_votes/filter_df.total_votes>=0.5)
filter_df_2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20A1J42RKSO1I|          1|           97|        105|   N|                N|
| R6HXKDRL0KIE4|          1|           45|         52|   N|                Y|
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
| R4UPLCUG1KGSA|          5|           25|         27|   N|                Y|
|R107YKUN3UFA9D|          5|           34|         36|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
#filter the data where all the rows are part of the vine program
vine_program_df=filter_df_2.filter(filter_df_2.vine=='Y')
vine_program_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1EVWPBBNRGKB2|          5|           65|         67|   Y|                N|
|R1U1DL7QBF8P9P|          4|           45|         47|   Y|                N|
| R4KEU66BD36CR|          5|           33|         34|   Y|                N|
|R37XROWTNIZV3T|          4|           37|         41|   Y|                N|
|R2XLAQL0E2UPWQ|          4|           33|         34|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
#filter the data where all the rows are not part of the vine program
no_vine_program_df=filter_df_2.filter(filter_df_2.vine=='N')
no_vine_program_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20A1J42RKSO1I|          1|           97|        105|   N|                N|
| R6HXKDRL0KIE4|          1|           45|         52|   N|                Y|
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
| R4UPLCUG1KGSA|          5|           25|         27|   N|                Y|
|R107YKUN3UFA9D|          5|           34|         36|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
#filter to just the 5 star reviews
five_star_reviews=filter_df_2.filter(filter_df_2.star_rating==5)
five_star_reviews.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
| R4UPLCUG1KGSA|          5|           25|         27|   N|                Y|
|R107YKUN3UFA9D|          5|           34|         36|   N|                Y|
|R1MPFLWI15CRJV|          5|          238|        242|   N|                N|
|R13GBBEGG9LB0U|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
#determine total number of reviews
print(filter_df_2.count())

37993


In [17]:
#determine the number of 5 star reviews
print(five_star_reviews.count())

20670


In [28]:
#determine the number of paid 5 star reviews 
paid_five_star = five_star_reviews.filter(five_star_reviews.vine=='Y')
print(paid_five_star.count())


65


In [29]:
#determine the number of unpaid 5 star reviews
no_pay_five_stars = five_star_reviews.filter(five_star_reviews.vine=='N')
print(no_pay_five_stars.count())

20605


In [30]:
#create variables
total_reviews = 37993
total_five_star_reviews = 20670
total_paid_five_star_reviews = 65
total_unpaid_five_star_reviews = 20605

In [31]:
#determine the percentage of unpaid five star reviews
unpaid_five_star_percent = (total_unpaid_five_star_reviews/total_five_star_reviews)*100
print(unpaid_five_star_percent)


99.68553459119497


In [32]:
#determine the percentage of paid five star reviews 
paid_five_star_percent = (total_paid_five_star_reviews/total_five_star_reviews)*100
print(paid_five_star_percent)

0.3144654088050315


In [33]:
#determine the percentage of 5 star reviews vs total reviews
five_star_percent = (total_five_star_reviews/total_reviews)*100
print(five_star_percent)

54.40475877135261
